## Shared Memory

### Learning objectives

In this lab we'll learn about shared memory. Shared memory is extremely fast, user managed, on-chip memory that can be used to share data between threads within a thread block. This can be used to manage data caches, speed up high-performance cooperative parallel algorithms, and facilitate global memory coalescing in cases where it would otherwise not be possible. We will learn:

- How to determine what algorithms might benefit from shared memory
- How to allocate and utilize shared memory in a CUDA kernel
- How to use CUDA synchronization primitives to facilitate shared memory usage

After completing this lab, you will be able to speed up algorithms such as one-dimensional or multi-dimensional stencils that involve data reuse, and you will know how to synchronize threads in a threadblock.

### Prerequisites

It is assumed that participants are aware of:

- How to launch CUDA kernels that use both blocks and threads


- Basic memory management (`cudaMalloc()`, `cudaFree()`, `cudaMemcpy`)


- The NVIDIA GPU memory hierarchy


- How to efficiently access global memory

## Shared Memory

Shared memory works a little differently from DRAM. It is a per-SM resource (from a hardware perspective), and logically (from a software perspective) it is a per-threadblock resource.

Some uses for shared memory:
- Inter-thread communication within a block
- Cache data to reduce redundant global memory accesses
- Improve upon global memory access patterns

Shared memory is also physically organized differently from DRAM, and that implies the optimal access pattern is different. We will return to this point later.

## 1D Stencil

Consider applying a 1D stencil to a 1D array of elements. A very simple stencil is one where each output element is the sum of input elements within a fixed radius. As an example, if the radius is 3, then each output element is the sum of 7 input elements:

![](images/1D_stencil_radius.png)



### Implementing Within a Block

If we wanted to apply this stencil entirely within a threadblock, a straightforward strategy is that each thread processes one output element. There are `blockDim.x` threads per block, so we will process `blockDim.x` elements of the array.

A key insight about this problem is that input elements are read several times. If `radius = 3`, then each input element is read (up to) seven times:

![](images/1D_stencil_reuse.png)



### Sharing Data Between Threads

Within a block, threads share data via **shared memory**. Shared memory is extremely fast, on-chip memory that is user managed (as opposed to hardware caches). Shared memory is typically also used as a cache, but one the developer controls.

Shared memory is declared using the <span style="font-family:courier;color:orange;">**\_\_shared\_\_**</span> qualifier, and is allocated on a per-block basis. Data written to a block's shared memory cannot be read by other blocks.

To cache data from the input array in this example, we:
- Read `(blockDim.x + 2 * radius)` input elements from device memory to shared memory
- Compute `blockDim.x` output elements
- Write `blockDim.x` output elements to device memory

In order to correctly implement this algorithm, each block needs a "halo" of `radius` elements at each boundary (otherwise the entries near the edges would not be able to compute the stencil correctly.

![](images/1D_stencil_halo.png)



### Stencil Kernel

Let's try writing the kernel for this stencil, now that we understand our strategy and the new syntax we need.

![](images/1D_stencil_kernel.png)

Note that we reserved a number of threads equal to `radius` to read the halo.



### Data Race

Unfortunately, our implementation above will not work. The depiction above implies that all threads operate simultaneously and that we can read the code sequentially to understand the control flow. However, this is not the case: in general, there is no prescribed order in CUDA in which threads execute. Your program may not assume any specific ordering in how threads execute their instructions, as the programming model does not enforce such a constraint.

To see why this is a problem, suppose (as a hypothetical; we are not saying such a scenario necessarily would occur) that thread 15 executes before all other threads.

![](images/data_race.png)

First, thread 15 stores index 18 from device memory into shared memory. This thread does not enter the conditional block since it is not one of the threads assigned to read, so it immediately proceeds to read in the data from shared memory. Now, one of the operations in the for loop that computes the stencil will be to add index 19 (which is part of the halo) from shared memory to our `result`. However, we posited that the thread responsible for filling that location in shared memory (thread 0) has not loaded this data yet. So the data is uninitialized, and we will obtain an incorrect result.

In order to be generally correct with respect to the CUDA programming model, we need some way to ensure that thread 0 has executed its stores to the shared memory halo before other threads can read from it.



### __syncthreads()

CUDA provides a way to address this race condition with the <span style="font-family:courier;color:orange;">**\_\_syncthreads()**</span> primitive. <span style="font-family:courier;">**\_\_syncthreads()**</span> (which has a void return type) synchronizes all threads within a block. In this way we can avoid common race conditions (read-after-write, write-after-read, and write-after-write hazards).

<span style="font-family:courier;">**\_\_syncthreads()**</span> requires that all threads in a block must reach the <span style="font-family:courier;">**\_\_syncthreads()**</span> statement in the kernel before any thread can continue. In this way it serves as a thread barrier, similar to barrier statements in other parallel programming models such as MPI and OpenMP.

Note that an important requirement of this primitive is that all threads in a block must be able to reach this statement (or else the kernel may hang), so be very careful when using it in conditional code.



### Revised Stencil Kernel

Now that we know how to avoid the data race, let's rewrite our kernel:

![](images/1D_stencil_kernel_revised.png)

## Exercise

Implement the 1D stencil application we've discussed above that uses shared memory. A skeleton code is provided in [exercises/stencil_1d.cu](exercises/stencil_1d.cu). Edit that file, paying attention to the FIXME locations. The code will verify output and report any errors.

In [ ]:
!nvcc -arch=native -o stencil_1d exercises/stencil_1d.cu; ./stencil_1d

If you get stuck, you can consult the solution in [solutions/stencil_1d.cu](solutions/stencil_1d.cu).

### Shared Memory Performance

Shared memory is organized in 32 **banks** which are 4 bytes wide, with successive 4-byte words belonging to different banks. One way to think about shared memory is that it is a two-dimensional array of memory whose columns correspond to banks. Since each bank is 4-bytes wide, the width of the array is 128 bytes. A given row contains one slot of each bank. The height of the array (number of rows) is as large as the shared memory requested by your code, up to a limit of 48 kB, the maximum you can request per block. Since each row represents all 32 banks, multiple locations in shared memory correspond to the same bank (for example bytes 0-3, 128-131, etc. are in bank 0, the first "column").

Shared memory can deliver one item from each bank in a transaction, and shared memory accesses are issued together for all 32 threads in a warp. So the shortest possible load operation is one that loads a 4-byte word from bank 0, another 4-byte word from bank 1, going up to bank 31. Conversely, if in the same transaction threads all read different words from the same bank (e.g. thread 0 reads bytes 0-3, thread 1 reads bytes 128-131, etc., which can be thought of as a columnar access pattern) then shared memory will serialize those transactions. The first cycle would service the first thread, the second cycle would service the second thread, etc. Generally, if N threads access different 4-byte words in the same bank, those N accesses are executed serially. This will result in extremely low performance. But if we always access in a row-based fashion (or more generally ensure there's no more than one item per bank read by the warp) then we'll get the maximum performance.

If N threads access the same word in the same transaction, that will run at full performance (the operation will be **multicast**), even though these are in the same bank. So this is different from DRAM: in DRAM we are focused on memory coalescing, but with shared memory we are interested in avoiding **bank conflicts**, which occur when multiple threads access separate words in the same bank. Keep that in mind going forward: the way we think about and optimize shared memory accesses will in general be different from the way we think about and optimize global memory accesses.

Shared memory is a bit challenging to grasp fully and you will probably need to experiment a bit to understand it fully. But let's look at some patterns of access activity to try to grasp this.

### Bank Addressing Examples

Let's first think about some cases without bank conflicts. On the left side, we have thread 0 accessing (either reading from or writing to) a word in bank 0 (which does not necessarily need to be bytes 0-3), thread 1 is accessing bank 1, etc. The right side shows a possible swizzling where threads access banks in some manner that is not a simple linear mapping, but nevertheless has every thread accessing a different bank. In both cases, because there are no bank conflicts, the access happens at maximum performance.

![](images/bank_addressing_examples.png)

Now let's look at a couple other cases. First, on the left side, we have at most two threads accessing any bank, and we call this a 2-way bank conflict. This runs half as fast as the cases above. On the right side, we have 16 threads accessing bank 0 and 16 threads accessing bank 8, and this runs 1/16 as fast as the cases above.

![](images/bank_addressing_examples_2.png)

### Avoiding Bank Conflicts

Returning to the idea of a columnar shared memory layout, let's think of shared memory as an array with a width of 32 (the number of banks). And to be concrete let's imagine a 32x32 array where the number of rows is 32. If threads in a warp access a row, there are no bank conflicts and the access happens as fast as possible. If threads access a column, for example all threads in a warp access bank 1, this is a 32-way bank conflict, and this will be the slowest possible access (since the requests serialize).

In the depiction below we have 32 warps, and each column represents one of those warps. Each square is a 4-byte location of shared memory, and the number in the square indicates the warp ID of the thread that is accessing that location. The color of the square indicates which bank it is in (look on the left for the key mapping colors to banks).

![](images/avoiding_bank_conflicts.png)

Is there anything we can do to improve this? A clever idea is to use **padding**. Usually we allocate shared memory as an array and we have control over how that array is allocated. If we allocate shared memory as a two-dimensional array, we can choose the number of rows and columns. Since the physical layout of shared memory never changes (it always has 32 "columns"), we can choose our array allocation to work optimally by padding the array so it has 33 columns instead of 32 (say). This effectively staggers the mapping of the logical array to physical locations in shared memory, and does not require us to change anything about the way we access shared memory -- we can access columns 0 through 31 as normal and ignore column 32 (just leave it unused). But now if our threads access a column of the array in the code, they are *not* all accessing the same column in shared memory -- they now access 32 different banks, and the bank conflicts are removed. It is important to remember this concept because sometimes the most logical way to write an algorithm is with a logical 2D array where you will access the array in both row-based and column-based patterns in the same algorithm (an example is [transposing a matrix using shared memory](https://devblogs.nvidia.com/efficient-matrix-transpose-cuda-cc/)), and you want to be able to do this without having to suffer a performance penalty for the columnar accesses or having to rewrite your algorithm in a complicated way to deal with it.

![](images/avoiding_bank_conflicts_2.png)

## Exercise

Let's apply shared memory to the 2D matrix multiply we looked at in lab 1. FIXME locations are provided in the skeleton code [exercises/matrix_mul_shared.cu](exercises/matrix_mul_shared.cu). See if you can successfully load the required data into shared memory and then appropriately update the dot product calculation.

In [ ]:
!nvcc -arch=native -o matrix_mul_shared exercises/matrix_mul_shared.cu; ./matrix_mul_shared

If you get stuck, you can consult the solution in [solutions/matrix_mul_shared.cu](solutions/matrix_mul_shared.cu).

## Exercise

A classic example demonstrating how to use shared memory is [transposing a matrix](https://en.wikipedia.org/wiki/Transpose). If you're unfamiliar with the matrix transpose operation, it means flipping a matrix -- which is just a two-dimensional array -- along its diagonal, so the element in position `(i, j)` becomes the element in position `(j, i)`. Think about how this works as a memory access operation. Suppose we have a 2D array stored in row-major format (elements in a given row correspond to contiguous locations in memory). Transposing a given row means turning it into a column, so if we implement the transpose by reading a row and writing to a column, then the read will be coalesced and the write will be uncoalesced. (If we flip the memory ordering, we just have the same problem in reverse.)

So we need a clever approach here to blunt the effect of the uncoalesced writes, which we know will perform suboptimally on a GPU. Shared memory can come to the rescue here. Our approach will be to make a copy of a tile of the matrix into shared memory, and then do an columnar *read* from shared memory so that the *write* (of a row) to global memory can be coalesced. Since the read of the input matrix was also coalesced, we achieve optimal global memory bandwidth in both directions. In effect, our algorithm will transpose a tile of the matrix (flip it across the diagonal) and then also transpose the matrix elements within that tile (flip it across the diagonal of that tile). In the image, `idata` would be our input data and `odata` would be our output data (the transposed matrix).

![](images/shared_memory_matrix_transpose.jpg)

For more on this, check out [An Efficient Matrix Transpose in CUDA C/C++](https://devblogs.nvidia.com/efficient-matrix-transpose-cuda-cc/).

The code [exercises/matrix_transpose.cu](exercises/matrix_transpose.cu) contains a skeleton code to tackle this problem. The matrices are allocated and initialized for you, and the code will check whether you solved the problem correctly. What you need to complete the implementation of are two kernels: `gmemTranspose`, which performs the inefficient transpose operation fully in global memory, and `smemTranspose`, which performs the transpose operation using shared memory. Note that we've defined a macro `IDX(ROW, COL)` which you can use to ensure your 2D indexing into a 1D array makes sense. Make sure you've obtained the correct results for both kernels before you start analyzing performance (below).

In [ ]:
!nvcc -arch=native -o matrix_transpose exercises/matrix_transpose.cu; ./matrix_transpose

(If you get stuck, you can check out a solution in [solutions/matrix_transpose.cu](solutions/matrix_transpose.cu).)

Now let's look at the performance of these kernels. `gmemTranspose` provides us a baseline: if shared memory was useful, the kernel using it (`smemTranspose`) should be faster. We can use the [Nsight Systems](https://developer.nvidia.com/nsight-systems) command line interface to collect basic statistics on the code, including the average kernel duration for each one. Execute the next code block and look at the CUDA Kernel Statistics section. Which was faster?

In [ ]:
!nsys profile --stats=true ./matrix_transpose

Let's get a little more detail on the performance of these two kernels. The [Nsight Compute](https://developer.nvidia.com/nsight-compute) command line interface can collect performance counters on these kernels. Let's run it and collect performance counters on each of our two kernels. We can use `--kernel-name` to select the kernel name (the compiled name of the kernel is the same name Nsight Systems reports). The `--launch-skip` option allows us to skip some number of kernel invocations (this is useful for skipping the first few in case they are longer as the device warms up), and the `--launch-count` option determines how many invocations we profile once we've skipped the first few. In this case we'll just profile a single invocation since the performance of each invocation should be fairly similar.

The number we want to pay close attention to is how well we're using DRAM. If you look for `SOL DRAM` in the `GPU Speed of Light` section, the percentage tells you what fraction of peak DRAM bandwidth we're achieving. If we can get DRAM bandwidth up to at least 70%, we've done a reasonably good job utilizing DRAM. If we're much lower than that, it's a sign that we're using DRAM ineffectively, for example not coalescing our accesses or not launching enough threads to saturate the device. What results do you see, and do they make sense given what we've discussed?

In [ ]:
!ncu --kernel-name regex:gmemTranspose --launch-skip 10 --launch-count 1 ./matrix_transpose

In [ ]:
!ncu --kernel-name regex:smemTranspose --launch-skip 10 --launch-count 1 ./matrix_transpose

There's one more optimization we can apply to the `smemTranspose` kernel. Based on the above image, our write from a row of global memory into a row of shared memory is contiguous and therefore achieves optimal shared memory bandwidth. But our read from shared memory is columnar, and as we have discussed, columnar accesses to shared memory incur a heavy performance penalty if the column corresponds to a single bank in shared memory. Is that the case here? Well, we said that the "width" of a row in shared memory is 32 entries of 4 bytes each (that is, 32 banks), so the width in bytes is 128. Since our matrix is (single-precision) floating point data, it just so happens that a row of 32 entries (the width of our tile) exactly matches the width in bytes of the row of shared memory. This means that our columnar access will indeed be all in the same bank and thus incur a 32-way bank conflict in each warp.

Fix this problem using the techniques we discussed earlier to ensure that the logical columnar access to the shared memory tile are not all in the same bank. After ensuring correctness of your code, re-run the above Nsight Systems and Nsight Compute profile collections to ensure the kernel got faster. Did DRAM efficiency improve?

## Is Shared Memory Necessary?

![](images/shared_memory_versus_l1.png)

## Review

In this lab we learned:

- Use <span style="font-family:courier;">**\_\_shared\_\_**</span> to declare a variable/array in shared memory
  - Data is shared between threads in a block
  - Not visible to threads in other blocks
  

- Use <span style="font-family:courier;">**\_\_syncthreads()**</span> as a barrier
  - Use to prevent data hazards
  
  
- We can use shared memory as a user-managed cache
  - When we do this, we want to pay attention to bank conflicts

## Further Study

[Shared Memory](https://devblogs.nvidia.com/using-shared-memory-cuda-cc/)


[CUDA Programming Guide](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#shared-memory)


[CUDA Runtime API Documentation](https://docs.nvidia.com/cuda/index.htmlhttps://docs.nvidia.com/cuda/cuda-runtime-api/index.html)

## Lab Materials

You can download this notebook using the `File > Download as > Notebook (.ipnyb)` menu item. Source code files can be downloaded from the `File > Download` menu item after opening them.